# Drive & Environment Setting

In [28]:
IN_COLAB = False

try:
  import google.colab
  IN_COLAB = True
except:
  pass

if IN_COLAB == True:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd /content/gdrive/MyDrive/Tesi
  PROJECT_PATH = '/content/gdrive/MyDrive/Tesi'
  #!ls
else:
  %cd /Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive
  PROJECT_PATH = '/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive'

import warnings
import pickle
import sys
import os

if IN_COLAB == True:
  sys.path.insert(0, os.path.abspath('functions'))
  sys.path.insert(0, os.path.abspath(''))
else:
  sys.path.insert(0, os.path.abspath('functions'))
  sys.path.insert(0, os.path.abspath(''))

warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential, Model
from keras.layers import Dense, Lambda
from keras.layers import Activation
from keras.activations import exponential, relu
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.layers import Input, Layer, InputSpec, Reshape
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
tf.config.run_functions_eagerly(True)
from scipy.optimize import minimize, differential_evolution

from pkl import store_data, load_data
from functions.compressor import compress
from functions.compute_target import compute_target

import random
from collections import defaultdict

import time



print("Libraries imported")

/Users/edoardorossi/Documents/Universita/Tesi/Tesi_GDrive
Libraries imported


# Import Anagrafica e CTA

In [25]:
cta30_def_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_def.csv')
CTA30_def = [cta30_def_path]
  
df_def = pd.read_csv(CTA30_def[0])

cta30_18_path = os.path.join(PROJECT_PATH, 'CTA30/DF_clean/clean_CTA_30_2018.csv')
CTA30_18 = [cta30_18_path]

df = pd.read_csv(CTA30_18[0])
df['VLV_RAF_FDBK_00'] = df['VLV_RAF_FDBK_00'] / 100
df['VLV_SUR_FDBK_00'] = df['VLV_SUR_FDBK_00'] / 100

scaler = MinMaxScaler()
act_f_m = df.loc[:,'F_M_FDBK_00']
a_f_m_reshaped = np.array(act_f_m).reshape(-1, 1)
a_f_m_scaled = scaler.fit_transform(a_f_m_reshaped)

min_f = np.min(a_f_m_scaled)
max_f = np.max(a_f_m_scaled)

a_raf_df = df['VLV_RAF_FDBK_00'] * 0.5
a_sur_df = df['VLV_SUR_FDBK_00'] * 0.5 + 0.5000000001

pca_df = compress(df)

# Learning

In [26]:
n = load_data('_.pkl')
if IN_COLAB == True:
    n = load_data('*.pkl')

action_vector_dimension = 2

# Architettura rete neurale
input_dim = n['n_components'] + action_vector_dimension
inner_dim_1 = 32
inner_dim_2 = 32
output_dim = 1

print("input dim: ", input_dim)
print("inner layer1 dim: ", inner_dim_1)
print("inner layer2 dim: ", inner_dim_2)
print("output dim: ", output_dim)

# Definizione rete neurale
rete_neurale = Sequential()
rete_neurale.add(Dense(inner_dim_1, input_dim=input_dim, activation='relu'))
rete_neurale.add(Dense(inner_dim_2, input_dim=inner_dim_1, activation='relu'))
rete_neurale.add(Dense(output_dim, input_dim=inner_dim_2, activation='relu'))

adam = Adam(learning_rate=0.001)
rete_neurale.compile(loss='mse', optimizer=adam, metrics=['mae'])

rete_neurale.load_weights('rete_neurale_DQN_1.h5')

# Definisco funzione obiettivo per la massimizzazione della predizione 
def objective(x):
    pca_s_prime = pca_df.loc[s_prime,:]
    nn_input_prime = np.concatenate((pca_s_prime.values.reshape(7,1), x[0].reshape(1,1), x[1].reshape(1,1)), axis=0)
    nn_input_prime = np.reshape(nn_input_prime, (1,input_dim))
    funct = rete_neurale.predict(nn_input_prime, verbose=0)[0][0]
    return -funct

input dim:  9
inner layer1 dim:  32
inner layer2 dim:  32
output dim:  1


In [16]:
[S, feat] = df.shape
current_state = 0
s = current_state
current_episode = 0
total_episodes = 10000

alpha = 0.1
gamma = 1

for current_episode in range(total_episodes):

    s_prime = s+1
    if s_prime > S-1:
      print("s =", s)
      break
    # a1 = frequenza sul motore di mandata
    a1 = df.loc[s,'F_M_FDBK_00']

    # a2 = percentuale di apertura delle valvole caldo/freddo (esclusive)
    a_raf = df.loc[s,'VLV_RAF_FDBK_00']
    a_sur = df.loc[s,'VLV_SUR_FDBK_00']
    if a_raf == 0:
      if a_sur == 0:
        a2 = 0
      else:
        a2 = a_sur_df[s]
    else:
      a2 = a_raf_df[s]
       
    a = (a1,a2)

    # r = reward 
    r = -df.loc[s,'PW_ATT_M_00']

    # Condizioni iniziali random per la minimizzazione
    data_f = a_f_m_scaled
    a0_f = data_f[np.random.randint(len(data_f))]
    a0_v = np.random.rand()
    x0 = [a0_f, a0_v]
  
    # Dominio della minimizzazione 
    bounds = [(min_f, max_f), (0,1)]
    print("minimizing...")
    start_time = time.time()
    # Risultato della minimizzazione
    objective_minimized = differential_evolution(objective, bounds=bounds)
    end_time = time.time()
    print("time elapsed: ", end_time - start_time, "seconds")
    print("minimized")

    print("predicting...")
    # Compongo input della predizione di q'
    a1_prime_input = np.array([objective_minimized.x[0]]).reshape(1,1)
    a2_prime_input = objective_minimized.x[1].reshape(1,1)
    s_prime_input = pca_df.loc[s_prime,:].values.reshape(7,1)
    conc1 = np.concatenate((s_prime_input, a1_prime_input), axis=0)
    a2_prime_input = np.expand_dims(a2_prime_input, axis=1).reshape(1,1)
    conc2 = np.concatenate((conc1, a2_prime_input), axis=0)
    nn_input_prime = np.reshape(conc2, (1,input_dim))
    # Compongo input della predizione di q
    a1_input = np.array([a1]).reshape(1,1)
    a2_input = np.array([a2]).reshape(1,1)
    s_input = pca_df.loc[s,:].values.reshape(7,1)
    conc1 = np.concatenate((s_input, a1_input), axis=0)
    a2_input = np.expand_dims(a2_input, axis=1).reshape(1,1)
    conc2 = np.concatenate((conc1, a2_input), axis=0)
    nn_input = np.reshape(conc2, (1,input_dim))

    # Predizione di q'
    q_prime = rete_neurale.predict(nn_input_prime, verbose = 0)[0]
    print("q' predicted:", q_prime)
    q_prime = q_prime[0]

    # Predizione di q
    q_ = rete_neurale.predict(nn_input, verbose = 0)[0] 
    print("q predicted:", q_)
    q_ = q_[0]

    # q = avr + gamma*max(q')
    q = q_ + alpha*(r + gamma*q_prime - q_)

    print("q: ", q)
    # Calcola il valore target per il training della rete neurale
    target = q
    target = np.array([target])

    # Aggiornamento della rete neurale
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiornamento  dello stato corrente
    s = s_prime

    # Plot statistiche ogni n episodi    
    n_print = 20
    if current_episode%n_print == 0:
      print("current_episode = ", current_episode)
      print("Episodio ",current_episode,"/",total_episodes)
      print("\n")



 




minimizing...
time elapsed:  54.28936719894409 seconds
minimized
predicting...
q' predicted: [0.5355783]
q predicted: [12.217145]
q:  11.048988300561906
current_episode =  0
Episodio  0 / 10000


minimizing...
time elapsed:  58.04374694824219 seconds
minimized
predicting...
q' predicted: [0.5334561]
q predicted: [10.777891]
q:  9.753447651863098
minimizing...
time elapsed:  45.2493040561676 seconds
minimized
predicting...
q' predicted: [0.53030604]
q predicted: [9.365186]
q:  8.481697767972946
minimizing...
time elapsed:  47.72534799575806 seconds
minimized
predicting...
q' predicted: [0.5273432]
q predicted: [7.9932656]
q:  7.246673387289047
minimizing...
time elapsed:  65.14515423774719 seconds
minimized
predicting...
q' predicted: [0.523759]
q predicted: [6.6684847]
q:  6.05401211977005
minimizing...
time elapsed:  45.07673192024231 seconds
minimized
predicting...
q' predicted: [0.5210131]
q predicted: [5.3978953]
q:  4.910207110643387
minimizing...
time elapsed:  50.66403818130493 

KeyboardInterrupt: 

In [30]:
[S, feat] = df.shape
current_state = 0
s = current_state
current_episode = 0
total_episodes = 10000

alpha = 0.1
gamma = 1

for current_episode in range(total_episodes):

    s_prime = s+1
    if s_prime > S-1:
      print("s =", s)
      break
    # a1 = frequenza sul motore di mandata
    a1 = df.loc[s,'F_M_FDBK_00']

    # a2 = percentuale di apertura delle valvole caldo/freddo (esclusive)
    a_raf = df.loc[s,'VLV_RAF_FDBK_00']
    a_sur = df.loc[s,'VLV_SUR_FDBK_00']
    if a_raf == 0:
      if a_sur == 0:
        a2 = 0
      else:
        a2 = a_sur_df[s]
    else:
      a2 = a_raf_df[s]
       
    a = (a1,a2)

    # r = reward 
    r = -df.loc[s,'PW_ATT_M_00']

    target = compute_target(min_f, max_f, objective, s_prime, 
                            a[1], a[2], s, r, alpha, gamma, 
                            rete_neurale, pca_df, input_dim)

    # Aggiornamento della rete neurale
    rete_neurale.fit(nn_input, target, epochs=1, verbose=0)

    # Aggiornamento  dello stato corrente
    s = s_prime

    # Plot statistiche ogni n episodi    
    n_print = 20
    if current_episode%n_print == 0:
      print("current_episode = ", current_episode)
      print("Episodio ",current_episode,"/",total_episodes)
      print("\n")



 




IndexError: tuple index out of range

In [17]:
rete_neurale.get_weights()

[array([[ 1.99308544e-01, -2.53150821e-01,  1.90691739e-01,
          6.80205226e-03, -9.76977870e-02,  2.69673318e-01,
          5.76299429e-02,  3.51702601e-01,  3.56871307e-01,
         -3.44222598e-02, -7.47099593e-02,  1.70016527e-01,
          3.25891733e-01,  4.90468740e-03, -9.05256867e-02,
         -3.32993805e-01, -2.96946734e-01,  1.44851893e-01,
          1.63999468e-01, -1.43457100e-01,  4.06609476e-02,
         -3.81809652e-01,  1.01603389e-01,  1.64344400e-01,
          1.16929471e-01, -8.14542472e-02,  3.19382668e-01,
         -2.25958675e-02,  3.34421217e-01,  1.83908571e-03,
          3.52562755e-01, -1.33883685e-01],
        [-3.07298005e-02, -1.21894583e-01,  2.64793247e-01,
         -2.45333239e-01,  1.97341945e-02, -1.76741064e-01,
          1.43447369e-01, -3.18900168e-01,  2.89489180e-01,
         -2.77845860e-01, -1.67392612e-01,  3.19974840e-01,
          3.23116094e-01, -2.75554359e-01,  2.05121905e-01,
         -3.72754693e-01, -1.18248723e-01,  5.20452484e-

In [18]:
q = rete_neurale.predict(nn_input, verbose = 0)[0]

In [19]:
q

array([0.], dtype=float32)

In [20]:
objective_minimized.x

array([0.00228734, 0.99852852])

current_state = 50

In [22]:
rete_neurale.save_weights('rete_neurale_DQN_1.h5')